### Chatbot logic and implementation

In [ ]:
import pandas as pd
import sqlite3
from datetime import datetime, timedelta


In [ ]:
class BookChat:
    def __init__(self):
        self.books = pd.read_csv("./data/books.csv")
        self.orders = pd.read_csv("./data/orders.csv")
        self.customers = pd.read_csv('./data/customers.csv')

    def reply(self, msg: str):
        print(f"\nBookChat: {msg}")

    def user_msg(self, msg: str):
        print(f"User: {msg}\n")

    def input(self, msg: str):
        self.reply(msg)
        user_input = input("\nUser: ")
        self.user_msg(user_input)
        return user_input


In [ ]:
class Books(BookChat):
    def __init__(self):
        super().__init__()

    def get_book_details(self, book_title: str):
        return self.books[self.books["title"] == book_title]

    def check_availability(self):
        user_input = self.input(msg="What book are you looking for?")
        self.reply(msg="One moment, let me check the inventory.")

        if user_input:
            book = self.books[self.books["title"].str.contains(user_input, case=False)]

            if book.empty:
                self.reply(
                    f"Sorry, it seems that we do not have any book with the title '{user_input}' in stock."
                )

                return []

            else:
                self.reply(
                    f"Yes, '{book.iloc[0]['title']}' by {book.iloc[0]['author']} is currently in stock."
                )

                return book.iloc[0]
        else:
            self.reply("Sorry, I did not catch that. Could you please start again?")
            self.check_stock()

    def check_inventory(self, book):
        self.user_msg("How many copies are available?")

        self.reply("One moment, let me check the inventory.")

        self.reply(f"We currently have {book['quantityInStock']} copies in stock.")

    def check_price(self, book):
        self.user_msg("What is the price of the book?")

        self.reply("Let me find that information for you...")

        self.reply(
            f"{book['title']} by {book['author']} is priced at ${book['price']}."
        )

    def reserve_copy(self, book, customer):
        self.user_msg("Can you reserve a copy for me?")

        # Get the maximum orderID value
        max_order_id = self.orders["orderID"].max()

        # Increment the maximum orderID value to get the next available ID
        next_order_id = max_order_id + 1

        # Update orders table with new orderID
        new_order = {
            "orderID": next_order_id,
            "bookID": book["bookID"],
            "customerID": customer['customerID'],
            "orderDate": datetime.now().strftime("%Y-%m-%d"),
            "pickupDate": (datetime.now() + timedelta(days=2)).strftime("%Y-%m-%d"),
            "status": "Reserved",
        }
        
        self.orders = pd.concat(
            [self.orders, pd.DataFrame([new_order])], ignore_index=True
        )
        self.orders.to_csv("./data/orders_DUMP.csv", index=False)

        # Reduce quantityInStock by 1
        self.books.loc[self.books["bookID"] == book["bookID"], "quantityInStock"] -= 1
        self.books.to_csv("./data/books_DUMP.csv", index=False)

        self.reply(
            f"Thank you {customer['firstName']}, I have reserved a copy of {book['title']} for you. Please visit the store within the next 48 hours to pick it up."
        )

In [ ]:
class Orders(BookChat):
    def __init__(self):
        super().__init__()

    def get_orders(self):
        return self.orders

In [ ]:
class Customers(BookChat):
    def __init__(self):
        super().__init__()

    def get_customer(self, customerID):
        return self.customers[self.customers["customerID"] == customerID].iloc[0]

    def add_customer(self):
        self.reply(
            "Welcome to Argo! We hope you have a great experience with us. To help us assist you better, please provide a few details about yourself."
        )
        # Get max customer ID
        max_id = self.customers["customerID"].max()

        # Ask for name, address, phone number, email
        firstName = self.input("What is your first name?")
        lastName = self.input("What is your last name?")
        phone = self.input("What is your phone number?")
        email = self.input("What is your email address? (Leave blank if none)")

        new_customer = {
            "customerID": max_id + 1,
            "firstName": firstName,
            "lastName": lastName,
            "phone": phone,
            "email": email,
        }

        self.customers = pd.concat(
            [self.customers, pd.DataFrame([new_customer])], ignore_index=True
        )

        self.customers.to_csv("./data/customers_DUMP.csv", index=False)

        self.reply(f"Thank you for shopping with us! Your customer ID is {max_id + 1}.")
        self.reply("Please use this customer ID for future purchases and queries.")
        self.reply(f"What can I help you with, {firstName}?")

        return self.get_customer(max_id + 1)

In [ ]:
chatbot = BookChat()
books = Books()
orders = Orders()
customers = Customers()

### Welcome

In [ ]:
chatbot.reply("Hello, I'm BookChat by Argo Bookstore!")

customer_type = chatbot.input(
    "Are you a new customer or an existing customer? (New/Existing)"
)

if "existing" in customer_type:
    customerInfo = customers.get_customer(
        int(
            chatbot.input(
                "Welcome back! Please provide your customer ID so that we can provide you with a personalized experience!"
            )
        )
    )

    chatbot.reply(f"Welcome back, {customerInfo['firstName']}! What can I help you with?")

else:
    customerInfo = customers.add_customer()

### Scenario 1: Book availablity and booking

In [ ]:
bookSearched = books.check_availability()

In [ ]:
books.check_inventory(bookSearched)
books.check_price(bookSearched)

In [ ]:
books.reserve_copy(bookSearched, customerInfo)